Before running:
- Double-check the Tableau workbook for accuracy and layout
- Make a new access token and update the 'tabcmd_token' environment variable (they expire quick)
- Download the 'Names to Export' sheet from the newest One Pager workbook to this folder
- If there's a folder called 'Reports' from the last time, rename or move it so it doesn't get overwritten.

The purpose of this notebook is to download a pdf for each student in the list and make each report available, both in a folder by advisory and as a single merged pdf. For best results run cells one by one.

If something goes wrong, look into the lines starting with ```system('tabcmd```.

In [ ]:
import pandas as pd
from os import system, path, walk, makedirs, environ

TOKEN_NAME = 'DonKALC'
TOKEN_VALUE = environ['tabcmd_token']
REPORT_NAME = 'One-PagerQ2'

In [ ]:
with open('Names to Export.csv', encoding='UTF-16') as file:
    names = pd.read_csv(file, sep='\t', names=['Student', 'Advisor', 'Type'])[1:]
    
names['Advisor'].unique()

In [ ]:
names = names[~names['Student'].str.contains("---")]

This cell will take a while; you can follow its progress in the terminal window that launched Jupyter, or by opening tabcmd.log. **Change your power settings to keep the computer from sleeping!** It's a good idea to check some pdfs as they come out.

In [ ]:
system(f'tabcmd login -s servername -t sitename --token-name {TOKEN_NAME} --token-value {TOKEN_VALUE}')

for _, (student, advisor, type) in names.iterrows():
    if not path.exists(r'Reports/' + advisor):
        makedirs(r'Reports/' + advisor)
    system(f'tabcmd export "One-PagerQ2/OnePager{type}Version?NameAndID="{student} --pdf -f "Reports/{advisor}/{student}.pdf"')

If the kernel freezes, use the cell below to restart from the middle of the loop.

In [ ]:
system(f'tabcmd login -s servername -t sitename --token-name {TOKEN_NAME} --token-value {TOKEN_VALUE}')

stopped_at = 'Student_100_id_100'
flag = False

for _, (student, advisor, type) in names.iterrows():
    if student == stopped_at:
        flag = True
    if flag:
        if not path.exists(r'Reports/' + advisor):
            makedirs(r'Reports/' + advisor)
        system(f'tabcmd export "One-PagerStaging_/OnePager{type}Version?NameAndID="{student} --pdf -f "Reports/{advisor}/{student}.pdf"')

Another variation, this time updating specific students after correcting their details:

In [ ]:
system(f'tabcmd login -s servername -t sitename --token-name {TOKEN_NAME} --token-value {TOKEN_VALUE}')

corrections = [id_1, id_2, id_3, id_4]

for _, (student, advisor, type) in names.iterrows():
    lasid = int(student.split('_')[1])
    if lasid in corrections:
        if not path.exists(r'Reports/' + advisor):
            makedirs(r'Reports/' + advisor)
        system(f'tabcmd export "One-Pager/OnePager{type}Version?NameAndID="{student} --pdf -f "Reports/{advisor}/{student}.pdf"')

This cell collects reports from all the directories into one file for easy printing.

In [ ]:
from pypdf import PdfReader, PdfWriter

writer = PdfWriter()

for subdir, dirs, files in walk('Reports'):
    if files:
        for file in files:
            reader = PdfReader(path.join(subdir, file))
            writer.add_page(reader.pages[0])
        writer.add_blank_page()

writer.write("Reports/Merged_OnePagers.pdf")